# Predictions

In [1]:
import sys
sys.path.insert(1, "../")

In [2]:
from biome.text import Pipeline
import spacy
import glob
from pathlib import Path
import helper
from tqdm import tqdm
import os
import numpy as np
import pandas as pd

In [3]:
nlp = spacy.load('es')
nlp.tokenizer = helper.custom_tokenizer(nlp)

# XLM-Roberta - char - lstm 

In [4]:
pl_trained = Pipeline.from_pretrained("submitted_models/xlmr-char-lstm/model.tar.gz")

In [5]:
pl_trained.trainable_parameters

279384232

In [22]:
name_and_size = [(name, p.numel()) for name, p in pl_trained._model.named_parameters()]

In [24]:
name_and_size

[('_head.backbone.embedder.token_embedder_char._embedding._module.weight',
  8576),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.weight_ih_l0',
  24576),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.weight_hh_l0',
  49152),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.bias_ih_l0',
  384),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.bias_hh_l0',
  384),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.weight_ih_l0_reverse',
  24576),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.weight_hh_l0_reverse',
  49152),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.bias_ih_l0_reverse',
  384),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.bias_hh_l0_reverse',
  384),
 ('_head.backbone.embedder.token_embedder_transformers._matched_embedder.transformer_model.embeddings.word_embeddings.weight

In [28]:
sum([size for name, size in name_and_size if '_head.backbone.encoder.' in name])

1181696

In [8]:
#txt_files = sorted(glob.glob("../data/dev-set1-to-publish/cantemist-ner/cc*.txt"))
txt_files = sorted(glob.glob("../data/test-background-set-to-publish/*.txt"))

In [8]:
output_dir = "predictions_for_test_set_xlmr-char-lstm"
for txt_file in tqdm(txt_files):
    ann_file = os.path.basename(txt_file).replace(".txt", ".ann")
    if os.path.isfile(os.path.join(output_dir, ann_file)):
        continue
    helper.text2ann(Path(txt_file), Path(output_dir), nlp, pl_trained, replace_tokens=False, split_sentences=True)

 23%|██▎       | 1220/5232 [45:55<3:49:01,  3.43s/it] 

Split:  ['La', 'valoración', 'analítica', 'mostraba', ':', 'glucosa', '92', 'mg', '/', 'dl', '(', '60', '-', '100', ')', ',', 'creatinina', '1', ',', '2', 'mg', '/', 'dl', '(', '0', ',', '8', '-', '1', ',', '3', ')', ',', 'urato', '7', ',', '9', 'mg', '/', 'dl', '(', '3', ',', '5', '-', '7', ',', '2', ')', ',', 'Na', ',', 'K', ',', 'Cl', ',', 'Calcio', 'y', 'fósforo', 'normales', ',', 'perfil', 'hepático', 'normal', ',', 'colesterol', 'total', '233', 'mg', '/', 'dl', '(', 'HDL', '52', 'mg', '/', 'dl', ',', 'LDL', '153', 'mg', '/', 'dl', ')', ',', 'triglicéridos', '152', 'mg', '/', 'dl', '(', '<', '150', ')', ',', 'proteínas', 'totales', '6', ',', '4', 'g', '/', 'dl', '(', '6', ',', '6', '-', '8', ',', '3', ')', ',', 'albúmina', '3', ',', '3', 'g', '/', 'dl', '(', '3', ',', '5', '-', '5', ',', '2', ')', ',', 'Fe', '63', 'μg', '/', 'dl', '(', '70', '-', '180', ')', ',', 'ferritina', '265', 'ng', '/', 'ml', '(', '20', '-', '400', ')', ',', 'saturación', 'de', 'transferina', '25', '%', '('

 59%|█████▉    | 3092/5232 [1:47:08<3:36:53,  6.08s/it] 

Split:  ['De', 'esta', 'manera', ',', 'estas', 'fuerzas', 'combinadas', 'pueden', 'relacionarse', 'incrementando', 'sus', 'acciones', ',', 'por', 'eso', 'en', 'lo', 'que', 'se', 'relaciona', 'a', 'la', 'prevención', 'de', 'UPP', 'utilizando', 'la', 'Escala', 'de', 'Braden', ',', 'se', 'observa', 'lo', 'mucho', 'que', 'estos', 'factores', 'pueden', 'reforzar', 'sus', 'acciones', 'cuando', 'están', 'asociados', ',', 'reduciendo', 'el', 'valor', 'cuantitativo', 'de', 'riesgo', 'la', 'incidencia', 'de', 'UPP.9', '\n', 'La', 'escala', 'de', 'Braden', 'cuenta', 'con', 'seis', 'subescalas', 'divididas', 'en', 'percepción', 'sensorial', '(', 'prueba', 'la', 'capacidad', 'de', 'reaccionar', 'de', 'forma', 'significativa', 'a', 'la', 'presión', 'relacionada', 'a', 'la', 'incomodidad', ')', ',', 'humedad', '(', 'se', 'refiere', 'al', 'nivel', 'en', 'que', 'la', 'piel', 'se', 'expone', 'a', 'la', 'humedad', ')', ',', 'actividad', '(', 'en', 'relación', 'con', 'el', 'grado', 'de', 'actividad', 'fís

100%|██████████| 5232/5232 [3:04:52<00:00,  2.12s/it]  


**Problematic file that contains a sentence too long for the model: ../data/test-background-set-to-publish/S0212-16112009000600018-1.txt**

In [14]:
# xcheck our metrics with their metrics
# !rm -r ../data/dev-set1-to-publish/cantemist-ner/.ipynb_checkpoints
# !rm -r ./$output_dir/.ipynb_checkpoints
# !python ../../cantemist-evaluation-library/src/main.py -g ../data/dev-set1-to-publish/cantemist-ner/ -p ./dev1_4 -s ner

rm: cannot remove '../data/dev-set1-to-publish/cantemist-ner/.ipynb_checkpoints': No such file or directory
rm: cannot remove './dev1_4/.ipynb_checkpoints': No such file or directory

-----------------------------------------------------
Clinical case name			Precision
-----------------------------------------------------
cc_onco100.ann		0.636
-----------------------------------------------------
cc_onco101.ann		0.833
-----------------------------------------------------
cc_onco106.ann		1.0
-----------------------------------------------------
cc_onco11.ann		0.848
-----------------------------------------------------
cc_onco114.ann		1.0
-----------------------------------------------------
cc_onco116.ann		0.87
-----------------------------------------------------
cc_onco117.ann		0.889
-----------------------------------------------------
cc_onco121.ann		1.0
-----------------------------------------------------
cc_onco123.ann		0.714
-----------------------------------------------------
c

# fasttext - char - lstm 

In [4]:
pl_trained = Pipeline.from_pretrained("submitted_models/fasttext-char-lstm/model.tar.gz")

In [5]:
pl_trained.trainable_parameters

8783556

In [6]:
name_and_size = [(name, p.numel()) for name, p in pl_trained._model.named_parameters()]

In [7]:
name_and_size

[('_head.backbone.embedder.token_embedder_word.weight', 4239900),
 ('_head.backbone.embedder.token_embedder_char._embedding._module.weight',
  6784),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.weight_ih_l0',
  24576),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.weight_hh_l0',
  49152),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.bias_ih_l0',
  384),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.bias_hh_l0',
  384),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.weight_ih_l0_reverse',
  24576),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.weight_hh_l0_reverse',
  49152),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.bias_ih_l0_reverse',
  384),
 ('_head.backbone.embedder.token_embedder_char._encoder._module._module.bias_hh_l0_reverse',
  384),
 ('_head.backbone.encoder._module.weight_ih_l0', 1138688),

In [13]:
sum([size for name, size in name_and_size if '_label_projection_layer' in name])

5125

In [12]:
pl_trained.config.as_dict()

{'name': 'candemist-ner-first-hpo',
 'tokenizer': {'lang': 'en',
  'max_sequence_length': None,
  'max_nr_of_sentences': None,
  'text_cleaning': None,
  'segment_sentences': False,
  'start_tokens': None,
  'end_tokens': None},
 'features': {'word': {'embedding_dim': 300,
   'lowercase_tokens': True,
   'trainable': True,
   'weights_file': None},
  'char': {'embedding_dim': 64,
   'lowercase_characters': True,
   'encoder': {'bidirectional': True,
    'hidden_size': 128,
    'num_layers': 1,
    'type': 'gru'},
   'dropout': 0.16517050992687604},
  'transformers': None},
 'head': {'dropout': 0.2689579604286324,
  'feedforward': None,
  'labels': ['O',
   'I-MORFOLOGIA_NEOPLASIA',
   'U-MORFOLOGIA_NEOPLASIA',
   'B-MORFOLOGIA_NEOPLASIA',
   'L-MORFOLOGIA_NEOPLASIA'],
  'type': 'biome.text.modules.heads.token_classification.TokenClassification'},
 'encoder': {'bidirectional': True,
  'hidden_size': 512,
  'input_size': 556,
  'num_layers': 1,
  'type': 'allennlp.modules.seq2seq_encoder

In [7]:
#txt_files = sorted(glob.glob("../data/dev-set1-to-publish/cantemist-ner/cc*.txt"))
txt_files = sorted(glob.glob("../data/test-background-set-to-publish/*.txt"))

In [8]:
output_dir = "predictions_for_test_set_fasttext-char-lstm"
for txt_file in tqdm(txt_files):
    ann_file = os.path.basename(txt_file).replace(".txt", ".ann")
    if os.path.isfile(os.path.join(output_dir, ann_file)):
        continue
    helper.text2ann(Path(txt_file), Path(output_dir), nlp, pl_trained, replace_tokens=False)

100%|██████████| 5232/5232 [21:57<00:00,  3.97it/s]
